In [1]:
import pandas as pd
import numpy as  np

from TargetEncoderv2 import TargetEncoder
from FeatureSelector import FeatureSelector

from sklearn.metrics import *
from sklearn.model_selection import *


In [2]:
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [4]:
ip_device_os_dict = {}
cnt = 0
for i, chunk in enumerate(pd.read_csv("../input/train.csv", dtype=dtypes, usecols=['ip', 'device', 'os'], chunksize=10000000)):
    print("processing chunk {}".format(i))
    tmp_dict = chunk.reset_index().set_index(['ip','device','os'])['index'].to_dict()
    ip_device_os_dict.update(tmp_dict)

for i, chunk in enumerate(pd.read_csv("../input/test.csv", dtype=dtypes, usecols=['ip', 'device', 'os'], chunksize=10000000)):
    print("processing chunk {}".format(i))
    tmp_dict = chunk.reset_index().set_index(['ip','device','os'])['index'].to_dict()
    ip_device_os_dict.update(tmp_dict)

print(len(ip_device_os_dict))
ip_device_os_dict = {k:i for i, k in enumerate(ip_device_os_dict)}


processing chunk 0
processing chunk 1
processing chunk 2
processing chunk 3
processing chunk 4
processing chunk 5
processing chunk 6
processing chunk 7
processing chunk 8
processing chunk 9
processing chunk 10
processing chunk 11
processing chunk 12
processing chunk 13
processing chunk 14
processing chunk 15
processing chunk 16
processing chunk 17
processing chunk 18
processing chunk 0
processing chunk 1
3669418


In [5]:
ip_device_os_series= pd.Series(ip_device_os_dict, name='ip_device_os')

In [7]:
#train.head().join(ip_device_os_series, on=['ip', 'device', 'os'], how='left')

In [8]:
orig_feats = ['ip', 'app', 'device', 'os', 'channel', 'weekofday', 'hourofday', 'is_attributed', 'ip_device_os']

val = pd.DataFrame(columns=orig_feats)
val.to_csv("../input/validation.csv", index=False)

tr = pd.DataFrame(columns=orig_feats)
tr.to_csv("../input/train_noval.csv", index=False)


    
for i, chunk in enumerate(pd.read_csv("../input/train.csv", dtype=dtypes, chunksize=5000000)):
    print("Proocessing chunk {}".format(i))
    chunk['click_time'] = pd.to_datetime(chunk['click_time'])
    chunk['weekofday'] = chunk['click_time'].dt.day
    chunk['hourofday'] = chunk['click_time'].dt.hour
    chunk = chunk.join(ip_device_os_series, on=['ip', 'device', 'os'], how='left')
    cond = (chunk['weekofday'] == 9) & (chunk['hourofday'].isin([4,5,9]))
    val = chunk.loc[cond]
    tr =chunk.loc[~cond]
    if len(val > 0):
        val[orig_feats].to_csv("../input/validation.csv", mode='a', header=False, index=False)
    if len(tr > 0):
        tr[orig_feats].to_csv("../input/train_noval.csv", mode='a', header=False, index=False)

Proocessing chunk 0
Proocessing chunk 1
Proocessing chunk 2
Proocessing chunk 3
Proocessing chunk 4
Proocessing chunk 5
Proocessing chunk 6
Proocessing chunk 7
Proocessing chunk 8
Proocessing chunk 9
Proocessing chunk 10
Proocessing chunk 11
Proocessing chunk 12
Proocessing chunk 13
Proocessing chunk 14
Proocessing chunk 15
Proocessing chunk 16
Proocessing chunk 17
Proocessing chunk 18
Proocessing chunk 19
Proocessing chunk 20
Proocessing chunk 21
Proocessing chunk 22
Proocessing chunk 23
Proocessing chunk 24
Proocessing chunk 25
Proocessing chunk 26
Proocessing chunk 27
Proocessing chunk 28
Proocessing chunk 29
Proocessing chunk 30
Proocessing chunk 31
Proocessing chunk 32
Proocessing chunk 33
Proocessing chunk 34
Proocessing chunk 35
Proocessing chunk 36


In [9]:
te = pd.DataFrame(columns=orig_feats)
te.to_csv("../input/test_noval.csv", index=False)

for i, chunk in enumerate(pd.read_csv("../input/test.csv", dtype=dtypes, chunksize=5000000)):
    print("Proocessing chunk {}".format(i))
    chunk['click_time'] = pd.to_datetime(chunk['click_time'])
    chunk['weekofday'] = chunk['click_time'].dt.day
    chunk['hourofday'] = chunk['click_time'].dt.hour
    chunk = chunk.join(ip_device_os_series, on=['ip', 'device', 'os'], how='left')

    chunk[orig_feats].to_csv("../input/test_noval.csv", mode='a', header=False, index=False)

Proocessing chunk 0
Proocessing chunk 1
Proocessing chunk 2
Proocessing chunk 3
